In [1]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import RandomRotation, RandomHorizontalFlip, RandomVerticalFlip

In [2]:
# data = {'image_path': [], 'label': []}

# for root, dirs, files in os.walk("data_"):
#     for file in files:
#         if file.endswith(".png"):
#             label = "left" if "L" in file.split('.')[0] else "right"
#             label = file.split('_')[0] +'_'+ label
#             data['image_path'].append(os.path.join(root, file))
#             data['label'].append(label)

# df = pd.DataFrame(data)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([
    RandomRotation(degrees=15),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [7]:
full_data = datasets.ImageFolder('data_')
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
test_loader = DataLoader(test_data, batch_size=2, shuffle=False)


In [8]:
vgg19 = models.vgg19(pretrained=True)
for param in vgg19.features.parameters():
    param.requires_grad = False

C:\Users\khang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\khang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
num_classes = len(full_data.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
vgg19 = vgg19.to(device)

# Định nghĩa hàm loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=0.001)


In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    vgg19.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg19(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    vgg19.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vgg19(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {accuracy}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
# torch.save(vgg19.state_dict(), "trained_model.pth")